## 과제 1

### **Q. 각 모델이 충족하는 속성에 대해 아래 표를 O/X로 채워주세요.**

📍5번째 속성은 **LSTM 기준으로** O/X 여부 판단해주세요 ! <br>
📍정답은 과제 마감 다음날 (9월 11일 수요일)에 **노션-정규세션-NLP basic**에 업로드 예정


> #### **속성 설명**
1. Order matters : 입력 시퀀스의 순서 중요 여부
2. Variable Length : 고정된 길이가 아닌 다양한 길이의 시퀀스를 처리할 수 있는지 여부
3. Differentiable : 미분가능
4. Pairwise encoding : 두 단어 사이의 관계를 표현
5. Preserves long-term : 장기적인 의존성


|               | N-gram | RNN   | LSTM  | Transformer |
|:-------------:|:------:|:-----:|:-----:|:-----------:|
| Order matters |      O  |  O   | O     | O           |
| Variable length |      |  O   | O     | O           |
| Differentiable |       |  O   | O     | O           |
| Pairwise encoding |    |     |      | O           |
| Preserves long-term |  |     | O     | O           |


## 과제 2


### 목표 : 독일어를 영어로 번역하는 모델 만들기
독일어 문장을 입력하면 영어로 번역해주는 모델을 seq2seq로 구현해봅시다

In [1]:
!pip install -U torchtext==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.9 MB/s eta 0:00:00


In [2]:
!python -m spacy download en
!python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 31.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [3]:
import numpy as np
import random
import time
import math
import spacy
from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator

import torch
import torch.nn as nn
import torch.optim as optim

### Tokenizers

- 문장의 토큰화, 태깅 등의 전처리를 수행하기 위해 `spaCy` 라이브러리에서 영어와 독일어 전처리 모듈을 설치해줍니다.
- 두 언어의 문장이 주어졌기 때문에 영어와 독일어 각각에 대해 전처리해주어야 합니다.


In [4]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [5]:
# 예시
result = spacy_en.tokenizer("I am a student.")

for i, token in enumerate(result):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: student
인덱스 4: .


필드(field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시해줍니다.

In [6]:
#===================================================
# 💡 토큰화 결과가 list로 반환될 수 있도록 return 결과값을 채워주세요
# seq2sxeq 논문에 의하면, input 단어의 순서를 바꾸면 최적화가 더 쉬워져 성능이 좋아진다고 합니다.
# 💡 독일어 토큰화 결과가 역순으로 return될 수 있도록 반영해주세요!
#===================================================
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어 문장을 spaCy를 사용해 토큰화하여 리스트로 반환
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

필드(field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시해줍니다.

In [7]:
# 독일어
SRC = Field(tokenize= tokenize_de, init_token = '<sos>', eos_token = '<eos>', lower = True)
# 영어
TRG = Field(tokenize= tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True)

### 데이터 불러오기

대표적인 영어-독어 번역 데이터셋 Multi30k을 불러옵니다.


In [8]:
!git clone https://github.com/multi30k/dataset.git

# 압축해제
!gunzip /content/dataset/data/task1/raw/train.de.gz
!gunzip /content/dataset/data/task1/raw/train.en.gz
!gunzip /content/dataset/data/task1/raw/val.de.gz
!gunzip /content/dataset/data/task1/raw/val.en.gz
!gunzip /content/dataset/data/task1/raw/test_2018_flickr.de.gz
!gunzip /content/dataset/data/task1/raw/test_2018_flickr.en.gz

Cloning into 'dataset'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 313 (delta 17), reused 21 (delta 16), pack-reused 281 (from 1)
Receiving objects: 100% (313/313), 18.21 MiB | 8.49 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [9]:
data_path = '/content/dataset/data/task1/raw/'

train_data = TranslationDataset(path=data_path, exts=('train.de', 'train.en'), fields=(SRC, TRG) )
val_data = TranslationDataset(path=data_path, exts=('val.de', 'val.en'), fields=(SRC, TRG) )
test_data = TranslationDataset(path=data_path, exts=('test_2018_flickr.de', 'test_2018_flickr.en'), fields=(SRC, TRG) )

In [10]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_data.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(val_data.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_data.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1071개


In [11]:
print(vars(train_data.examples[0]))
print(vars(train_data.examples[1]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}
{'src': ['.', 'antriebsradsystem', 'ein', 'bedienen', 'schutzhelmen', 'mit', 'männer', 'mehrere'], 'trg': ['several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']}


- `build_vocab`함수를 이용하여 영어와 독일어의 단어 사전을 생성해줍니다. 이를 통해 각 token이 indexing됩니다
- 단, vocabulary는 훈련 데이터셋에 대해서만 만들어져야 합니다.
- `min_freq`를 사용하여 최소 2번 이상 나오는 단어들만 사전에 포함되도록 합니다.

In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [13]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi[""]) # : 0
print(TRG.vocab.stoi[""]) # : 0
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
0
0
4112
1752


- 시퀀스 데이터는 각 문장의 길이가 다를 수 있습니다.
- `BucketIterator는` 유사한 길이를 가진 샘플들을 같은 배치에 묶어주는 역할을 하기 때문에, 고정된 길이로 맞추기 위한 패딩의 양을 최소화할 수 있습니다.

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

- 첫 번째 배치를 출력한 결과, [sequence length, batch size]라는 tensor가 생성됩니다
- `sequence length`는 해당 배치 내에서 가장 긴 문장의 길이를 의미하며, 이보다 짧은 문장은 <pad> token으로 채워집니다.
- 편의상 transpose한 뒤, 첫 번째와 두 번째 문장의 텐서를 출력하면, 특정 단어에 대응하는 인덱스가 출력되는 것을 알 수 있습니다.


In [15]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치의 text 크기: {src.shape}")
    src = src.transpose(1,0)
    print(src[0])
    print(src[1])

    break

첫 번째 배치의 text 크기: torch.Size([27, 128])
tensor([   2,    4,    0,    8,  147,   37,   14,   10, 1477,   54,   76,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1], device='cuda:0')
tensor([   2,    4,   28, 1931,   59,    0,   44, 5136,  109,  182,  113,    8,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1], device='cuda:0')


### Building the Seq2Seq with LSTM Model

- seq2seq 이해를 위한 과제이니, 아래를 참고하여 작성해도 무방합니다 :)


https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_LSTM_Tutorial.ipynb

### Encoder

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, p):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = nn.Dropout(p)

        #=========================================#
        # 💡아래줄에 embedding과 multi-layer LSTM 부분을 채워주세요 (dropout 포함)
        #=========================================#
        self.embedding =  nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=p)

    def forward(self, x):
        # x = [x length, batch size]
        embedding = self.dropout(self.embedding(x))  # embedding = [x length, batch size, emb size]

        outputs, (hidden, cell) = self.rnn(embedding)

        # hidden = [n layers, batch size, hid dim]
        # cell = [n layer, batch size, hid dim]
        # outputs = [src len, batch size, hid dim]

        return hidden, cell

### Decoder

In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, p):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = nn.Dropout(p)

        #=========================================#
        # 💡아래 코드를 채워주고, 각각 어떤 역할을 하는지 주석으로 간단히 설명해주세요
        #
        #=========================================#
        # Embedding: 원-핫 인코딩말고 특정 차원의 임베딩, 즉 dense vector로 변환
        self.embedding = nn.Embedding(output_dim, emb_dim)
        # LSTM: 임베딩된 벡터를 입력받아 hidden state와 cell state를 처리
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=p)
        # Fully connected layer: LSTM의 출력을 vocabulary 크기에 맞춰 예측값으로 변환
        self.fc = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden, cell):

        # 현재 input 형태 = [batch size]
        # Decoder는 한번에 하나의 토큰만 처리하도록 sequence length = 1이 되어야 합니다
        input = input.unsqueeze(0)

        embedding = self.dropout(self.embedding(input))

        #=========================================#
        # 💡self.rnn() 괄호 안 부분을 채워주세요
        #=========================================#
        output, (hidden, cell) = self.rnn(embedding, (hidden, cell))

        prediction = self.fc(output.squeeze(0))  #prediction = [batch size, output dim]

        return prediction, hidden, cell

### Seq2Seq

In [18]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        #=========================================#
        # 💡trg를 사용하여 decoder에 입력할 첫번째 input을 설정해주세요
        #=========================================#
        input = trg[0, :]

        for t in range(1, trg_len):

            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output

            # predictions들 중에 가장 잘 예측된 token 추출
            best_guess = output.argmax(1) # [batch size]

            input = trg[t] if random.random() < teacher_forcing_ratio else best_guess

        return outputs

### **Q. 위 코드에서는 매 시점마다 확률이 가장 높은 다음 단어를 선택하는 Greedy decoding  방식이 사용됩니다. 이런 방법을 채택할 경우 발생할 수 있는 문제점은 무엇일지 작성해주세요.**


```python

# predictions들 중에 가장 잘 예측된 token 추출
best_guess = output.argmax(1) # [batch size]

```


➡️ Greedy decoding은 매 시점마다 가장 확률이 높은 단어를 선택하는 방식이다. 이 방식은 각 시점에서 최적의 선택을 하지만, 전체 문맥에서는 그 선택이 적합하지 않을 수 있다. 따라서 장기적인 최적의 결과를 놓칠 수 있다. 이로 인해 문법적으로 부적절하거나 자연스럽지 않은 문장이 생성될 가능성이 있다.


### Train

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

모델 초기 가중치 값은 논문의 내용대로 U(−0.08,0.08)의 연속균등분포로부터 얻습니다.

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [21]:
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

### Evaluate

In [22]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 3
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 45s
	Train Loss: 5.049 | Train PPL: 155.842
	 Val. Loss: 5.115 |  Val. PPL: 166.478
Epoch: 02 | Time: 0m 44s
	Train Loss: 4.465 | Train PPL:  86.952
	 Val. Loss: 4.706 |  Val. PPL: 110.556
Epoch: 03 | Time: 0m 42s
	Train Loss: 4.143 | Train PPL:  62.999
	 Val. Loss: 4.539 |  Val. PPL:  93.557


In [27]:
model.load_state_dict(torch.load('/content/tut1-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

<ipython-input-27-806c4c390395>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/tut1-model.pt'))


| Test Loss: 5.133 | Test PPL: 169.522 |
